## Extract Vitals

## Imports

In [ ]:
import sys
sys.path.append("..")

import pandas as pd

import config
from cyclops.querier import SQLQuerier

## Load config, run query

In [ ]:
cfg = config.read_config("../configs/default/*.yaml")
querier = SQLQuerier(cfg)
data = querier.run_query()

## Browse through data

In [ ]:
pd.set_option('display.max_columns', None)
data

In [ ]:
data['measurement_name'].nunique(), data['measurement_name'].value_counts(), data['genc_id'].nunique(), data['measurement_code'].value_counts()